In [3]:
# Currently using the model `all-MiniLM-L6-v2` but embeddinggemma-300m from google can also be used
# https://huggingface.co/google/embeddinggemma-300m

In [17]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

## Alternative way

In [21]:
# Download and initialize the model
model_name = "all-MiniLM-L6-v2"
embedder = SentenceTransformer(model_name)

# Generate embeddings
sentences = ['I need a backend developer with JavaScript experience', 'I have 6 years of experience in FastAPI']
sentence_embeddings = embedder.encode(sentences)

print('Embeddings shape:')
print(embeddings.shape) # These embeddings has the type numpy.ndarray

# Calculate the similarity
similarity_score = F.cosine_similarity(torch.tensor(sentence_embeddings[0]), torch.tensor(sentence_embeddings[1]), dim=0)

print('Similarity Score:')
print(similarity_score)

Embeddings shape:
(2, 384)
Similarity Score:
tensor(0.2542)


## Generate Embeddings

In [4]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


In [5]:
# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')


In [14]:
# Sentences we want sentence embeddings for
sentences = ['I need a backend developer with JavaScript experience', 'I have 6 years of experience in FastAPI']

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print(sentence_embeddings.shape)

torch.Size([2, 384])


## Calculate Similarities

In [7]:
F.cosine_similarity(sentence_embeddings[0], sentence_embeddings[1], dim=0)

tensor(0.2542)

## Tensor <=> List (Conversion)

In [11]:
# Convert a tensor to list
embeddings_arr = sentence_embeddings[0].tolist()
print(embeddings_arr[:10])

[-0.11848553270101547, -0.10213883221149445, 0.011694430373609066, 0.054409824311733246, 0.003648617072030902, -0.03829596936702728, 0.014635247178375721, 0.045787375420331955, -0.03635245934128761, -0.0899975448846817]


In [12]:
# Convert the list into the tensor
embeddings = torch.tensor(embeddings_arr)
print(embeddings[:10])

tensor([-0.1185, -0.1021,  0.0117,  0.0544,  0.0036, -0.0383,  0.0146,  0.0458,
        -0.0364, -0.0900])
